# Apply igraph layout to CX network

This notebook applies an igraph layout to a CX network.

The network can be imported from NDEx or from a ```.cx``` file.

In [20]:
import ndex2
import igraph
import json

### Import a network

Import a network from NDEx or from a ```.cx``` file.

If you're importing from NDEx:
* ```IMPORT_FROM_NDEX``` should be set to ```True```. 
* ```NDEX_SERVER``` should be set to the address of the server you are importing from (usually ```ndexbio.org```).
* ```NDEX_USER``` should be the user name of your NDEx account.
* ```NDEX_PASSWORD``` should be the password of your NDEx account.
* ```NDEX_UUID``` should be the UUID of the network you are importing.

If you're importing from a ```.cx``` file:
* ```IMPORT_FROM_NDEX``` should be set to ```False```. 
* ```CX_FILE``` should be set to the path of the ```.cx``` file you are importing.

In [52]:
#Import network from NDEx or cx file
IMPORT_FROM_NDEX = False

#Constants for importing from ndex
NDEX_SERVER = 'dev.ndexbio.org'
NDEX_USER = 'sol015'
NDEX_PASSWORD = 'test12345'
NDEX_UUID = 'e4f080cd-1b49-11e9-a05d-525400c25d22'

#Constants for importing from cx file
CX_FILE = '/Users/sophieliu/Downloads/Adipogenesis (1).cx'

if IMPORT_FROM_NDEX:
    nice_cx = ndex2.create_nice_cx_from_server(NDEX_SERVER, username=NDEX_USER, password=NDEX_PASSWORD, uuid=NDEX_UUID)
else:
    nice_cx = ndex2.create_nice_cx_from_file(CX_FILE)
    
#Convert the nice cx file to an igraph graph
ig_network = igraph.Graph()

#Add nodes
for node_id, node in nice_cx.get_nodes():
    ig_network.add_vertex(name=str(node_id))

#Add edges
for edge_id, edge in nice_cx.get_edges():
    ig_network.add_edge(edge['s'], edge['t'])

### Apply a layout

Compute an igraph layout using one of the ```LAYOUT``` names found here: 

https://igraph.org/python/doc/tutorial/tutorial.html#layout-algorithms

Or here:

https://igraph.org/python/doc/tutorial/visualisation.html#graph-layouts (the list starts after scrolling down a little)

The ```SCALE``` variable represents the amount by which the size of the layout will be scaled up (with a value of ```1``` representing no scaling). 

The IGraph layout algorithm will lay nodes out in a square which is too small for most CX networks, so the layout must be scaled up to ensure that nodes do not heavily overlap. Most layouts will look good with ```SCALE``` set to a value between ```200``` and ```500```.

In [58]:
#Apply layout
LAYOUT = 'tree'
SCALE = 300

layout = ig_network.layout(LAYOUT)

#Transfer layout back to nice cx network
cartesian_layout = []
for node_index in range(len(ig_network.vs)):
    cartesian_layout.append({
        'node': int(ig_network.vs[node_index]['name']),
        'x': layout[node_index][0] * SCALE,
        'y': -layout[node_index][1] * SCALE
    })
nice_cx.set_opaque_aspect(ndex2.constants.CARTESIAN_LAYOUT_ASPECT, cartesian_layout)

### Export the network

The network will be exported in the same format in which it was imported (either to NDEx or as a ```.cx``` file).

If you are overwriting the imported network, ```REPLACE_OLD_NETWORK``` should be set to ```True```.

If you would like to make a copy instead, ```REPLACE_OLD_NETWORK``` should be set to ```False```.

In [59]:
#Export network
REPLACE_OLD_NETWORK = False

if IMPORT_FROM_NDEX:
    if REPLACE_OLD_NETWORK:
        nice_cx.update_to(NDEX_UUID, NDEX_SERVER, NDEX_USER, NDEX_PASSWORD)
    else:
        nice_cx.upload_to(NDEX_SERVER, NDEX_USER, NDEX_PASSWORD)
else:
    cx = nice_cx.to_cx()
    if REPLACE_OLD_NETWORK:
        with open(CX_FILE, 'w') as outfile:
            json.dump(cx, outfile)
    else:
        #Create new file name
        new_file_name = CX_FILE[:-3] + '_layout_' + LAYOUT + '_scale_' + str(SCALE) + '.cx'
        with open(new_file_name, 'w') as outfile:
            json.dump(cx, outfile)

Generating CX
